In [16]:
import os
import requests
import yfinance as yf

from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company. Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        company = yf.Ticker(symbol)
        return company.info


class CompanyNeesTool(BaseTool):
    name = "CompanyNews"
    description = """
    Use this to gether news of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        company = yf.Ticker(symbol)
        return company.news


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        company = yf.Ticker(symbol)
        return company.income_stmt


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        company = yf.Ticker(symbol)
        return company.history(period="1mo")

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        # StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyNeesTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock(symbol: NET), considering its financials, income statements, stock performance, and news. Help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CompanyOverview` with `{'symbol': 'NET'}`


{'address1': '101 Townsend Street', 'city': 'San Francisco', 'state': 'CA', 'zip': '94107', 'country': 'United States', 'phone': '888 993 5273', 'website': 'https://www.cloudflare.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premise, software-as-a-service applications, and IoT devices; and website and application security products comprising web application firewall, bot management, distributed denial of service, API gateways, SSL/TLS encry

{'input': "Give me financial information on Cloudflare's stock(symbol: NET), considering its financials, income statements, stock performance, and news. Help me analyze if it's a potential good investment.",
 'output': "### Cloudflare, Inc. (Symbol: NET) - Financial Overview\n\n#### Company Overview\n- **Industry**: Software - Infrastructure\n- **Sector**: Technology\n- **Market Cap**: $28.51 billion\n- **Employees**: 3,902\n- **Website**: [cloudflare.com](https://www.cloudflare.com)\n\nCloudflare provides a range of cloud-based security and performance solutions for businesses, including web application firewalls, DDoS protection, and content delivery networks. The company serves various sectors, including technology, healthcare, and finance.\n\n#### Financials\n- **Current Price**: $83.43\n- **Previous Close**: $79.43\n- **52-Week Range**: $53.88 - $116.00\n- **Beta**: 1.098 (indicating volatility relative to the market)\n- **Forward P/E Ratio**: 97.01\n- **Price to Sales (TTM)**: 19

In [3]:
import yfinance as yf

msft = yf.Ticker("MSFT") # microsoft

# get historical market data
hist = msft.history(period="1mo")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-09-05 00:00:00-04:00,407.619995,413.100006,406.130005,408.390015,14195500,0.0,0.0
2024-09-06 00:00:00-04:00,409.059998,410.649994,400.799988,401.700012,19609500,0.0,0.0
2024-09-09 00:00:00-04:00,407.239990,408.649994,402.149994,405.720001,15295100,0.0,0.0
2024-09-10 00:00:00-04:00,408.200012,416.329987,407.700012,414.200012,19594300,0.0,0.0
2024-09-11 00:00:00-04:00,415.500000,423.989990,409.579987,423.040009,19266900,0.0,0.0
2024-09-12 00:00:00-04:00,423.309998,427.369995,419.750000,427.000000,17418800,0.0,0.0
2024-09-13 00:00:00-04:00,425.829987,431.829987,425.459991,430.589996,15874600,0.0,0.0
2024-09-16 00:00:00-04:00,430.600006,433.529999,428.220001,431.339996,13834700,0.0,0.0
2024-09-17 00:00:00-04:00,440.230011,441.850006,432.269989,435.149994,18874200,0.0,0.0


In [4]:
# show financials:
# - income statement
msft.income_stmt


,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Tax Effect Of Unusual Items,-99918000.0,-2850000.0,43754000.0,180160797.164637
Tax Rate For Calcs,0.182,0.19,0.131,0.138266
Normalized EBITDA,133558000000.0,105155000000.0,99905000000.0,83831000000.0
Total Unusual Items,-549000000.0,-15000000.0,334000000.0,1303000000.0
Total Unusual Items Excluding Goodwill,-549000000.0,-15000000.0,334000000.0,1303000000.0
Net Income From Continuing Operation Net Minority Interest,88136000000.0,72361000000.0,72738000000.0,61271000000.0
Reconciled Depreciation,22287000000.0,13861000000.0,14460000000.0,11686000000.0
Reconciled Cost Of Revenue,74114000000.0,65863000000.0,62650000000.0,52232000000.0
EBITDA,133009000000.0,105140000000.0,100239000000.0,85134000000.0
EBIT,110722000000.0,91279000000.0,85779000000.0,73448000000.0


In [5]:
msft.quarterly_income_stmt


,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Tax Effect Of Unusual Items,-16263853.211009,-33660000.0,-29160000.0,-20700000.0,20855987.382214,NaN
Tax Rate For Calcs,0.191339,0.18,0.18,0.18,0.187892,NaN
Normalized EBITDA,34416000000.0,33741000000.0,33556000000.0,31845000000.0,28972000000.0,NaN
Total Unusual Items,-85000000.0,-187000000.0,-162000000.0,-115000000.0,111000000.0,NaN
Total Unusual Items Excluding Goodwill,-85000000.0,-187000000.0,-162000000.0,-115000000.0,111000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,22036000000.0,21939000000.0,21870000000.0,22291000000.0,20081000000.0,NaN
Reconciled Depreciation,6380000000.0,6027000000.0,5959000000.0,3921000000.0,3874000000.0,NaN
Reconciled Cost Of Revenue,19684000000.0,18505000000.0,19623000000.0,16302000000.0,16795000000.0,NaN
EBITDA,34331000000.0,33554000000.0,33394000000.0,31730000000.0,29083000000.0,NaN
EBIT,27951000000.0,27527000000.0,27435000000.0,27809000000.0,25209000000.0,NaN


In [7]:
# - balance sheet
msft.balance_sheet


,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Ordinary Shares Number,7434000000.0,7432000000.0,7464000000.0,7519000000.0
Share Issued,7434000000.0,7432000000.0,7464000000.0,7519000000.0
Net Debt,33315000000.0,12533000000.0,35850000000.0,43922000000.0
Total Debt,67127000000.0,59965000000.0,61270000000.0,67775000000.0
Tangible Book Value,121660000000.0,128971000000.0,87720000000.0,84477000000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,75531000000.0,111256000000.0,104749000000.0,130256000000.0
Other Short Term Investments,57216000000.0,76552000000.0,90818000000.0,116032000000.0
Cash And Cash Equivalents,18315000000.0,34704000000.0,13931000000.0,14224000000.0
Cash Equivalents,6744000000.0,26226000000.0,5673000000.0,6952000000.0


In [8]:
msft.quarterly_balance_sheet


,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Ordinary Shares Number,7434000000.0,7432305794.0,7431612107.0,7431000000.0,7432000000.0,NaN
Share Issued,7434000000.0,7432305794.0,7431612107.0,7431000000.0,7432000000.0,NaN
Net Debt,33315000000.0,45808000000.0,56914000000.0,NaN,12533000000.0,21648000000.0
Total Debt,67127000000.0,79911000000.0,88374000000.0,84989000000.0,59965000000.0,NaN
Tangible Book Value,121660000000.0,105161000000.0,89441000000.0,144029000000.0,128971000000.0,NaN
...,...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,75531000000.0,80013000000.0,80982000000.0,143945000000.0,111256000000.0,NaN
Other Short Term Investments,57216000000.0,60379000000.0,63677000000.0,63493000000.0,76552000000.0,NaN
Cash And Cash Equivalents,18315000000.0,19634000000.0,17305000000.0,80452000000.0,34704000000.0,NaN
Cash Equivalents,6744000000.0,8053000000.0,7769000000.0,73074000000.0,26226000000.0,NaN


In [9]:
# - cash flow statement
msft.cashflow








,2024-06-30,2023-06-30,2022-06-30,2021-06-30,2020-06-30
Free Cash Flow,74071000000.0,59475000000.0,65149000000.0,56118000000.0,NaN
Repurchase Of Capital Stock,-17254000000.0,-22245000000.0,-32696000000.0,-27385000000.0,NaN
Repayment Of Debt,-29070000000.0,-2750000000.0,-9023000000.0,-3750000000.0,NaN
Issuance Of Debt,29645000000.0,0.0,0.0,NaN,0.0
Issuance Of Capital Stock,2002000000.0,1866000000.0,1841000000.0,1693000000.0,NaN
Capital Expenditure,-44477000000.0,-28107000000.0,-23886000000.0,-20622000000.0,NaN
End Cash Position,18315000000.0,34704000000.0,13931000000.0,14224000000.0,NaN
Beginning Cash Position,34704000000.0,13931000000.0,14224000000.0,13576000000.0,NaN
Effect Of Exchange Rate Changes,-210000000.0,-194000000.0,-141000000.0,-29000000.0,NaN
Changes In Cash,-16179000000.0,20967000000.0,-152000000.0,677000000.0,NaN


In [10]:
msft.quarterly_cashflow


,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Free Cash Flow,23322000000.0,20965000000.0,9118000000.0,20666000000.0,19827000000.0,NaN
Repurchase Of Capital Stock,-4210000000.0,-4213000000.0,-4000000000.0,-4831000000.0,-5704000000.0,NaN
Repayment Of Debt,-13065000000.0,-11589000000.0,-2916000000.0,-1500000000.0,-1000000000.0,NaN
Issuance Of Debt,5447000000.0,6352000000.0,10773000000.0,25765000000.0,NaN,0.0
Issuance Of Capital Stock,534000000.0,522000000.0,261000000.0,685000000.0,512000000.0,NaN
Capital Expenditure,-13873000000.0,-10952000000.0,-9735000000.0,-9917000000.0,-8943000000.0,NaN
End Cash Position,18315000000.0,19634000000.0,17305000000.0,80452000000.0,34704000000.0,NaN
Beginning Cash Position,19634000000.0,17305000000.0,80452000000.0,34704000000.0,26562000000.0,NaN
Effect Of Exchange Rate Changes,-103000000.0,-80000000.0,72000000.0,-99000000.0,-81000000.0,NaN
Changes In Cash,-1216000000.0,2409000000.0,-63219000000.0,45847000000.0,8223000000.0,NaN


In [11]:
# show news
msft.news


[{'uuid': '613297f1-2459-3350-95d0-232e056abcae',
  'title': 'Where Will Lumen Technologies Stock Be in 1 Year?',
  'publisher': 'Motley Fool',
  'link': 'https://finance.yahoo.com/m/613297f1-2459-3350-95d0-232e056abcae/where-will-lumen-technologies.html',
  'providerPublishTime': 1728239400,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/tWBHLMbBujgyOP4cmcBLfQ--~B/aD03NDM7dz0xNDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/7ecc12669833056faa708977223977b6',
     'width': 1400,
     'height': 743,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/c79JdR8QGDAreXoD7LUFFg--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/7ecc12669833056faa708977223977b6',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['LUMN', 'MSFT']},
 {'uuid': '5798086c-0e02-34b1-b12f-7667c2bdc9d5',
  'title': 'Microsoft Corporation (MSFT) Mai

In [14]:
# show major holders
# msft.major_holders

# show institutional holders
msft.institutional_holders

,Date Reported,Holder,pctHeld,Shares,Value
0,2023-06-30,Vanguard Group Inc,0.0879,653247546,222456919314
1,2023-06-30,Blackrock Inc.,0.0722,536245027,182612881494
2,2023-06-30,State Street Corporation,0.0395,293614078,99987338122
3,2023-06-30,"FMR, LLC",0.0283,210111250,71551285075
4,2023-06-30,Price (T.Rowe) Associates Inc,0.0215,159904219,54453782738
5,2023-06-30,"Geode Capital Management, LLC",0.0199,148129470,50444009713
6,2023-06-30,Morgan Stanley,0.0166,123498322,42056118573
7,2023-06-30,JP Morgan Chase & Company,0.0152,113024837,38489477991
8,2022-12-31,Norges Bank Investment Management,0.0116,86316926,20700525193
9,2023-06-30,Capital World Investors,0.0115,85507261,29118642660
